In [4]:
from typing import TypedDict, Optional
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from docx import Document
import pandas as pd
import os

In [5]:
# --- 🔎 상태 정의 ---
class ProposalState(TypedDict, total=False):
    brand_name: Optional[str]
    brand_info: Optional[str]
    client_needs: Optional[str]
    recommended_media: Optional[str]
    previous_campaigns: Optional[str]
    proposal_text: Optional[str]
    proposal_file_path: Optional[str]

AgentState = ProposalState

In [6]:
# --- .env 에서 OPENAI API 키 불러오기 ---
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key)
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [7]:
# --- 🗄️ Tool 구성 (여기서는 예시 Stub 형태로 Tool 설정) ---
# 실제 서비스에서는 아래 Tool들을 LangChain Toolkit으로 구현

def db_query_tool(query: str) -> str:
    return f"[DB QUERY RESULT for: {query}]"

def web_search_tool(query: str) -> str:
    return f"[WEB SEARCH RESULT for: {query}]"

def vectordb_search_tool(query: str, collection_name: str, top_k: int = 3) -> str:
    vectorstore = Chroma(
        collection_name=collection_name,
        embedding_function=embedding
    )
    results = vectorstore.similarity_search(query, k=top_k)
    return "\n\n".join([doc.page_content for doc in results])

In [12]:
# --- ChromaDB에 campaign_media 데이터 올리기 (최초 1회) ---
csv_path = "../data/campaign_media.csv"
df = pd.read_csv(csv_path)

def row_to_text(row):
    return (
        f"캠페인 ID: {row['campaign_id']}, "
        f"매체 ID: {row['media_id']}, "
        f"시작일: {row['start_date']}, "
        f"종료일: {row['end_date']}, "
        f"구좌 수: {row['slot_count']}, "
        f"집행 가격: {row['executed_price']}, "
        f"진행 상태: {row['campaign_media_status']}"
    )

texts = df.apply(row_to_text, axis=1).tolist()

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents(texts)

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    collection_name="campaign_media_chroma"
)

In [ ]:
# --- Node 1: 브랜드 정보 + 고객 요구사항 분석 ---
def analyze_brand_and_needs(state: ProposalState):
    brand_name = state["brand_name"]
    # 브랜드 & 영업 기록 조회
    brand_query = f"""
    SELECT * FROM brand WHERE brand_name = '{brand_name}';
    SELECT * FROM sales_log WHERE brand_name = '{brand_name}' ORDER BY contact_time DESC LIMIT 1;
    """
    brand_info = db_query_tool(brand_query)

    # 고객 요구사항
    needs_query = f"""
    SELECT client_needs_summary FROM sales_log 
    WHERE brand_name = '{brand_name}' ORDER BY contact_time DESC LIMIT 1;
    """
    client_needs = db_query_tool(needs_query)

    return {**state, "brand_info": brand_info, "client_needs": client_needs}

# --- Node 2: 유사 집행 사례 조회 사진 정보 먼저---
def retrieve_previous_campaigns(state: ProposalState):
    client_needs = state.get("client_needs") or "옥외 광고 집행 사례"

    collection_name = "campaign_media_chroma"  # 벡터스토어 컬렉션 이름
    similar_cases = vectordb_search_tool(client_needs, collection_name)

    return {**state, "previous_campaigns": similar_cases}

# --- Node 3: 매체 추천 및 매칭 유사 집행 사례와 기존 매체 통합해서 MZ 패키지 매체 여러개 ---
# media, 웹검색?!
def recommend_media(state: ProposalState):
    client_needs = state.get("client_needs") or ""
    db_results = db_query_tool("SELECT * FROM media WHERE quantity > 0;")

    # ✅ 여기에 collection_name 추가!
    media_collection = "media_chroma"  # media 테이블을 chroma에 넣으셨다면 이 이름
    try:
        vector_results = vectordb_search_tool(client_needs, collection_name=media_collection)
    except:
        vector_results = "[VectorDB media_chroma가 아직 구축되지 않았습니다.]"

    web_results = web_search_tool(client_needs + "에 적합한 옥외 광고 매체 추천")

    combined = f"""
    [DB 추천]\n{db_results}\n
    [VectorDB 추천]\n{vector_results}\n
    [웹 검색 추천]\n{web_results}
    """
    return {**state, "recommended_media": combined}

# --- Node 4: 제안서 생성 (Word 파일 포함) ---
def generate_proposal(state: ProposalState):
    prompt = ChatPromptTemplate.from_template("""
    브랜드명: {brand_name}
    브랜드 정보: {brand_info}
    고객 요구사항: {client_needs}
    유사 집행 사례: {previous_campaigns}
    추천 매체: {recommended_media}

    위 정보를 바탕으로 고객에게 보낼 옥외 광고 제안서를 작성하세요.
    """)

    chain = prompt | llm
    proposal = chain.invoke(state).content

    # Word 파일 생성
    doc = Document()
    doc.add_heading(f"{state['brand_name']} 옥외 광고 제안서", level=1)
    doc.add_paragraph(proposal)

    file_name = f"{state['brand_name']}_제안서.docx"
    doc.save(file_name)

    return {**state, "proposal_text": proposal, "proposal_file_path": file_name}


In [17]:
# --- 🔗 그래프 구성 ---
graph = StateGraph(ProposalState)

graph.add_node("AnalyzeBrandAndNeeds", analyze_brand_and_needs)
graph.add_node("RecommendMedia", recommend_media)
graph.add_node("RetrievePreviousCampaigns", retrieve_previous_campaigns)
graph.add_node("GenerateProposal", generate_proposal)

graph.set_entry_point("AnalyzeBrandAndNeeds")
graph.add_edge("AnalyzeBrandAndNeeds", "RetrievePreviousCampaigns")
graph.add_edge("RetrievePreviousCampaigns", "RecommendMedia")
graph.add_edge("RecommendMedia", "GenerateProposal")
graph.set_finish_point("GenerateProposal")

proposal_graph = graph.compile()

In [18]:
# --- 🚀 실행 예시 ---
initial_state = {
    "brand_name": "라네즈",
}

final_state = proposal_graph.invoke(initial_state)

print("✅ 최종 제안서:\n")
print(final_state["proposal_text"])
print(f"📄 제안서 Word 파일 경로: {final_state['proposal_file_path']}")

✅ 최종 제안서:

**라네즈 옥외 광고 제안서**

---

**회사명:** 라네즈  
**작성일:** 2023년 10월 5일  
**작성자:** [작성자 이름]  

---

**1. 브랜드 정보**

라네즈는 혁신적인 스킨케어와 화장품 브랜드로, 높은 품질과 과학적인 접근을 통해 고객들에게 피부 관리의 새로움을 제공합니다. 고객의 피부 건강과 자연스러운 아름다움을 강조하며, 다양한 제품 라인을 제공합니다.  

---

**2. 최근 고객 요구사항 요약**

고객의 요구사항을 기반으로, 라네즈는 더 많은 브랜드 인지도를 확보해야 하며, 특히 특정 타겟층에 도달할 수 있는 효과적인 광고가 필요합니다. 이를 통해 새로운 고객층을 확보하고 브랜드 충성도를 강화하려고 합니다.  

---

**3. 유사 집행 사례**

아래는 유사한 목표와 범위의 과거 및 예정된 캠페인 사례입니다:  

- **캠페인 ID: 29**
  - 매체 ID: 10
  - 기간: 2025-07-10 ~ 2025-07-30
  - 구좌 수: 16
  - 집행 가격: 6,800,000 원
  - 진행 상태: 예정

- **캠페인 ID: 29**
  - 매체 ID: 5
  - 기간: 2025-07-05 ~ 2025-08-05
  - 구좌 수: 15
  - 집행 가격: 14,500,000 원
  - 진행 상태: 예정

- **캠페인 ID: 19**
  - 매체 ID: 33
  - 기간: 2025-05-10 ~ 2025-05-30
  - 구좌 수: 18
  - 집행 가격: 8,300,000 원
  - 진행 상태: 완료

---

**4. 추천 매체**

우리의 데이터베이스와 외부 검색 결과를 통해 추천된 옥외 광고 매체는 다음과 같습니다:

- **DB 추천:** [비어 있는 매체 리스트와 수량의 데이터베이스에 기반한 최적의 매체]
- **VectorDB 추천:** [기타 데이터베이스 관련 추가적인 매체]
- **웹 검색 추천:** 고객 요구사항에 적합한 매체 (옥외 광고 매체의 최신 트렌드와